## Import Dependencies


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import pandas as pd
import os
from PIL import Image

## Create Dataset for local Directories

In [9]:
class CIFAKE_Dataset(Dataset):
    def __init__(self, root_dir, transform=torchvision.transforms.ToTensor()):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['REAL', 'FAKE']
        
    def __len__(self):
        total_len = 0
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            total_len += len(os.listdir(class_dir))
        return total_len
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            class_files = os.listdir(class_dir)
            if idx < len(class_files):
                img_name = os.path.join(class_dir, class_files[idx])
                image = Image.open(img_name)
                label = self.classes.index(cls)
                if self.transform:
                    image = self.transform(image)
                return image, label
            else:
                idx -= len(class_files)

### Load the dataset

In [10]:
# Load the dataset from local directory without preprocessing
training_data = CIFAKE_Dataset(root_dir='data/train')
testing_data = CIFAKE_Dataset(root_dir='data/test')
# select only 30% of the dataset with torch.utils.data.Subset
training_data = torch.utils.data.Subset(training_data, torch.randperm(len(training_data))[:int(len(training_data))])
testing_data =  torch.utils.data.Subset(testing_data,  torch.randperm(len(testing_data)) [:int(len(testing_data) )])

image_datasets = {'train': training_data, 'val': testing_data}

In [11]:
# Defind the data loader
train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=64, shuffle=True)
dataloaders = {'train': train_loader, 'val': test_loader}

In [12]:
num_classes = 2
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes),
    nn.Softmax(dim=1)
)

/home/beik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/beik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [14]:
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [15]:
with open('resnet50_finetuned.log', 'w') as f:
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = corrects.double() / len(image_datasets[phase])
            
            print(  '{} Loss: {:.4f} Acc: {:.4f}'  .format(phase, epoch_loss, epoch_acc))
            f.write('{} Loss: {:.4f} Acc: {:.4f}\n'.format(phase, epoch_loss, epoch_acc))

# Save the trained model
torch.save(model.state_dict(), 'resnet50_finetuned.pth')

train Loss: 0.5587 Acc: 0.7558
val Loss: 0.5184 Acc: 0.7894
train Loss: 0.5117 Acc: 0.7935
val Loss: 0.5001 Acc: 0.8068
train Loss: 0.5034 Acc: 0.8012
val Loss: 0.4922 Acc: 0.8139
train Loss: 0.4974 Acc: 0.8072
val Loss: 0.4865 Acc: 0.8203
train Loss: 0.4936 Acc: 0.8112
val Loss: 0.4887 Acc: 0.8193
train Loss: 0.4913 Acc: 0.8127
val Loss: 0.4825 Acc: 0.8224
train Loss: 0.4895 Acc: 0.8143
val Loss: 0.4857 Acc: 0.8175
train Loss: 0.4876 Acc: 0.8160
val Loss: 0.4764 Acc: 0.8299
train Loss: 0.4850 Acc: 0.8193
val Loss: 0.4760 Acc: 0.8282
train Loss: 0.4841 Acc: 0.8196
val Loss: 0.4765 Acc: 0.8275
